In [1]:
import polars as pl
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pl.read_csv('for_polars.csv')
df = df.with_columns(
    pl.col('BaseDateTime')
    .str.to_datetime('%Y-%m-%d %H:%M:%S')
)
df.shape, type(df)

((2312579, 10), polars.dataframe.frame.DataFrame)

In [3]:
df.head()

,BaseDateTime,MMSI,LAT,LON,SOG,COG,Navigation_sequence,i,j
i64,datetime[μs],f64,f64,f64,f64,f64,i64,f64,f64
0,2021-02-27 12:52:00,2.05553e8,26.10166,-80.00492,6.5,329.0,1,7.0,15.0
1,2021-02-27 12:53:00,2.05553e8,26.10342,-80.00615,6.5,328.0,1,7.0,15.0
2,2021-02-27 12:54:00,2.05553e8,26.10527,-80.00736,6.5,328.1,1,7.0,15.0
3,2021-02-27 12:55:00,2.05553e8,26.10697,-80.00869,6.2,321.0,1,7.0,14.0
4,2021-02-27 12:56:00,2.05553e8,26.10697,-80.00869,6.2,321.0,1,7.0,14.0


In [4]:
df = df.with_row_count('Index')

In [5]:
df = df.with_columns(Encountering = pl.lit(0),
                Conflicting = pl.lit(0),
                TCPA = pl.lit(0),
                DCPA = pl.lit(0)
               )

In [6]:
df.head()

Index,,BaseDateTime,MMSI,LAT,LON,SOG,COG,Navigation_sequence,i,j,Encountering,Conflicting,TCPA,DCPA
u32,i64,datetime[μs],f64,f64,f64,f64,f64,i64,f64,f64,i32,i32,i32,i32
0,0,2021-02-27 12:52:00,2.05553e8,26.10166,-80.00492,6.5,329.0,1,7.0,15.0,0,0,0,0
1,1,2021-02-27 12:53:00,2.05553e8,26.10342,-80.00615,6.5,328.0,1,7.0,15.0,0,0,0,0
2,2,2021-02-27 12:54:00,2.05553e8,26.10527,-80.00736,6.5,328.1,1,7.0,15.0,0,0,0,0
3,3,2021-02-27 12:55:00,2.05553e8,26.10697,-80.00869,6.2,321.0,1,7.0,14.0,0,0,0,0
4,4,2021-02-27 12:56:00,2.05553e8,26.10697,-80.00869,6.2,321.0,1,7.0,14.0,0,0,0,0


In [7]:
import geopy.distance

def euclidian_distance(x1 = 0, y1 = 0, x2 = 0, y2 = 0):
    coord_1 = (x1, y1)
    coord_2 = (x2, y2)
    return geopy.distance.geodesic(coord_1, coord_2).m

In [8]:
from math import sin, cos, atan2, radians, degrees

def calculate_true_bearing(lon1, lat1, lon2, lat2, cog):
    
    '''
    # Convert latitudes and longitudes from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Difference in the longitudes
    dLon = lon2 - lon1
    
    # Calculate the bearing
    x = sin(dLon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dLon)
    initial_bearing = atan2(x, y)
    
    # Convert from radians to degrees
    initial_bearing = degrees(initial_bearing)
    
    # Normalize the bearing
    true_bearing = (initial_bearing + 360) % 360
    
    '''
    V_r_x = lon1 - lon2
    V_r_y = lat1 - lat2
    
    # Angle between vector R joining 2 ships and y axis
    angle = atan2(V_r_x, V_r_y) 
    
    true_bearing = (degrees(angle - cog) + 360) % 360
    
    return radians(true_bearing) 

In [9]:
def relative_velocity(sog1, cog1, sog2, cog2):
    
    sog1 *= 0.514
    sog2 *= 0.514
    
    cog1 = radians(cog1)
    cog2 = radians(cog2)
    
    V_own_x = sog1 * cos(cog1)
    V_own_y = sog1 * sin(cog1)  
    V_other_x = sog2 * cos(cog2)
    V_other_y = sog2 * sin(cog2)
    
    V_r_x = V_own_x - V_other_x
    V_r_y = V_own_y - V_other_y
    
    angle = atan2(V_r_x, V_r_y)
    mod = np.sqrt(V_r_x**2 + V_r_y**2)
    
    return angle, mod

In [18]:
import polars as pl

# Assuming df is your DataFrame

def calculate_metrics_for_pairs(data):
    # Perform a cross join to create ship pairs
    # Note: Adjust 'ship_id' and other relevant columns to your schema
    pairs = data.join(data, how="cross", suffix="_other")
    
    # Calculate distance and other metrics for each pair
    # Replace 'calculate_distance' with your actual distance calculation function
    pairs = pairs.with_columns([
        euclidian_distance(pairs['LON'], pairs['LAT'], pairs['LON_other'], pairs['LAT_other']).alias('distance')
        # Add other metrics calculations here
    ])
    
    # Filter pairs based on your criteria, e.g., distance
    close_pairs = pairs.filter(pl.col('distance') <= 1)
    
    # Calculate DCPA, TCPA, etc., for close_pairs and return the result
    # This step depends on your specific calculation functions
    return close_pairs

# Group by 'BaseDateTime' and apply the function
result = df.groupby('BaseDateTime').apply(calculate_metrics_for_pairs)

# This 'result' DataFrame now contains metrics for ship pairs that meet your criteria within each timestamp group


PanicException: UDF failed: the truth value of a Series is ambiguous

Hint: use '&' or '|' to chain Series boolean results together, not and/or. To check if a Series contains any values, use `is_empty()`.

In [16]:
df.groupby('BaseDateTime').apply(calculate_metrics_for_pairs)

PanicException: UDF failed: the truth value of a Series is ambiguous

Hint: use '&' or '|' to chain Series boolean results together, not and/or. To check if a Series contains any values, use `is_empty()`.

In [17]:
'''
This time we wont do much complicated thing as before 
(for a unique trajectory checek other ships sailing cole by at each time stamp) this what we were doing last time to check encounters 
Now It will be little simpler for each time stamp check encounters thats it.
'''

store = []
for i in range(df['BaseDateTime'].n_unique()):               # For a unique timestamp
    time_stamp = df['BaseDateTime'].unique()[i]
    temp_df = df.filter(pl.col('BaseDateTime') == time_stamp)    
    
    pairs = temp_df.join(temp_df, how='cross', suffix-'_other')
    
    pairs = pairs.with_columns([
        euclidian_distance(pairs['LON'], pairs['LAT'], pairs['LON_other'], pairs['LAT_other']).alias('distance')
    ])
    
    close_pairs = pairs.filter(pl.col('distance') <= 1)
    
    

    for j in range(temp_df.shape[0]):
        self_ship = temp_df[j]
        for k in range(j + 1, temp_df.shape[0]):
            target_ship = temp_df[k]
            
            x1, y1 = self_ship['LON'][0], self_ship['LAT'][0]
            x2, y2 = target_ship['LON'][0], target_ship['LAT'][0]
            R = euclidian_distance(x1, y1, x2, y2)/1000                 # distance between 2 ships in Kms
            
            if R <= 1:            # Mark the ship as encountering
                
                X_R, V_R = relative_velocity(self_ship['SOG'][0], self_ship['COG'][0], target_ship['SOG'][0], target_ship['COG'][0])
                X_os = radians(self_ship['COG'][0])
                theta_t = calculate_true_bearing(x1, y1, x2, y2, X_os)

                DCPA = R * sin((X_R - X_os - theta_t) - np.pi)
                TCPA = R * cos((X_R - X_os - theta_t) - np.pi) / V_R
                
                store.append(self_ship['Index'][0], target_ship['Index'][0], TCPA, DCPA)

SyntaxError: positional argument follows keyword argument (1151808070.py, line 12)

In [69]:
df.filter(pl.col('TCPA') > 0).n_unique()

1611

In [70]:
df.filter(pl.col('Encountering') > 0).n_unique()

2547

In [69]:
time_stamp = enc_df['BaseDateTime'].unique()[10]
temp_df = enc_df.filter(pl.col('BaseDateTime') == time_stamp)
temp_df

Index,,BaseDateTime,MMSI,LAT,LON,SOG,COG,Navigation_sequence,i,j,Encountering,Conflicting,TCPA,DCPA
u32,i64,datetime[μs],f64,f64,f64,f64,f64,i64,f64,f64,i32,i32,i32,i32
282957,282957,2021-01-20 15:00:00,3.38084457e8,26.09543,-80.11679,4.7,328.3,296,6.0,2.0,7,0,0,0


In [ ]:
for sequence in resampled_data['Navigation_sequence'].unique():
    
    navi_df = resampled_data[resampled_data['Navigation_sequence'] == sequence]
    
    for timestamp in navi_df['BaseDateTime'][::3]:
        self_ship = navi_df[navi_df['BaseDateTime'] == timestamp]
        time_df = resampled_data[resampled_data['BaseDateTime'] == timestamp]
        i,j = self_ship['i'], self_ship['j']
        time_df['i'] -= float(i)
        time_df['j'] -= float(j)
        
        # filtering data to just small 3x3 grid surrounding our ship with ship in middel on grid
        filtered_df = time_df[(time_df['i'].abs() <= 1) & (time_df['j'].abs() <= 1)]
        filtered_df = filtered_df.drop(self_ship.index)

        for mmsi in filtered_df['MMSI']:
            target_ship = filtered_df[filtered_df['MMSI'] == mmsi]
            c1 = (self_ship['LAT'].iloc[0], self_ship['LON'].iloc[0])
            c2 = (target_ship['LAT'].iloc[0], target_ship['LON'].iloc[0])
            dist = euclidian_distance(c1, c2)/1000        # calculating distance between self and target ship
            
            # check if Within the circle of observation of self ship
            if dist <= 1 :        
                resampled_data.loc[self_ship.index, 'Encountering'] = 1
                resampled_data.loc[target_ship.index, 'Encountering'] = 1
                
                # check for conflict
                # How ?? 
                Vs_x = self_ship['SOG'].iloc[0]*np.cos(np.radians(self_ship['COG'].iloc[0]))
                Vs_y = self_ship['SOG'].iloc[0]*np.sin(np.radians(self_ship['COG'].iloc[0]))
                
                Vt_x = target_ship['SOG'].iloc[0]*np.cos(np.radians(target_ship['COG'].iloc[0]))
                Vt_y = target_ship['SOG'].iloc[0]*np.sin(np.radians(target_ship['COG'].iloc[0]))
                
                Xs, Ys = convert_to_xy(self_ship['LAT'].iloc[0], self_ship['LON'].iloc[0])
                Xt, Yt = convert_to_xy(target_ship['LAT'].iloc[0], target_ship['LON'].iloc[0])      # x and will be in meters
                
                dist_next = np.sqrt((Xs + Vs_x*0.5144/60 - Xt - Vt_x*0.5144/60)**2 + (Ys + Vs_y*0.5144/60 - Yt - Vt_y*0.5144/60)**2)

                TCA = -1*((Xs - Xt)*(Vs_x - Vt_x) + (Ys - Yt)*(Vs_y - Vt_y)) / (Vs_x - Vt_x)**2 + (Vs_y - Vt_y)**2
                
                if TCA > 0 and dist_next < 1 :
                    resampled_data.loc[self_ship.index, 'Conflicting'] = 1
                    resampled_data.loc[target_ship.index, 'Conflicting'] = 1

## Some one time processing for modifying data

In [2]:
df = pl.read_csv('100lenTraj.csv')

df = df.with_columns(
    pl.col('BaseDateTime')
    .str.to_datetime('%Y-%m-%d %H:%M:%S')
)

In [3]:
df.shape, type(df)

((693295, 9), polars.dataframe.frame.DataFrame)

In [4]:
df.head()

BaseDateTime,MMSI,LAT,LON,SOG,COG,Navigation_sequence,i,j
datetime[μs],i64,f64,f64,f64,f64,i64,f64,f64
2021-01-01 02:21:38,745180,26.0371,-80.11735,7.3,3.5,1,0.0,2.0
2021-01-01 02:22:54,745180,26.03947,-80.11685,7.9,9.2,1,0.0,2.0
2021-01-01 02:24:03,745180,26.04155,-80.11638,6.2,12.3,1,0.0,2.0
2021-01-01 02:25:14,745180,26.04373,-80.11584,5.6,9.7,1,0.0,3.0
2021-01-01 02:26:23,745180,26.04484,-80.11564,3.0,8.6,1,1.0,3.0


In [5]:
# chceck the faluty sequence
# the faulty sequence has large time span but has only few data points
# according to COLREGs if ship is anchored it will give reading in every 3 min 
# so if for any seq dt/n_data_pts > 3 min we assume it to be faulty and we drop it

error_sequence = []
for seq in df['Navigation_sequence'].unique():
    temp_df = df.filter(pl.col('Navigation_sequence') == seq)
    no_dpts = temp_df.shape[0]
    dt = (temp_df['BaseDateTime'].max() - temp_df['BaseDateTime'].min()).total_seconds()/60
    if dt/no_dpts >= 6:
        error_sequence.append(seq)

In [6]:
# if we keep dt/npts limt to 4
    # so we found around 2250 unique navigation sequence to have some fault. 
    # these seq account for 372939 datapoints in AIS data which is 46% of total datapoints and 990 MMSI
    # we cant directly drop them
# if lmt is 6


len(error_sequence)

2167

In [7]:
df.filter(pl.col('Navigation_sequence') == error_sequence[1])

BaseDateTime,MMSI,LAT,LON,SOG,COG,Navigation_sequence,i,j
datetime[μs],i64,f64,f64,f64,f64,i64,f64,f64
2021-03-05 12:48:07,209470000,26.07003,-80.11542,0.0,245.4,4,3.0,3.0
2021-03-05 12:51:07,209470000,26.07035,-80.11515,1.2,15.9,4,3.0,3.0
2021-03-05 12:54:09,209470000,26.07207,-80.1149,2.9,5.9,4,4.0,3.0
2021-03-05 12:55:15,209470000,26.07302,-80.1148,3.3,5.1,4,4.0,3.0
2021-03-05 12:56:24,209470000,26.0741,-80.1147,3.5,4.4,4,4.0,3.0
2021-03-05 12:57:36,209470000,26.07527,-80.11463,3.6,1.5,4,4.0,3.0
2021-03-05 12:58:46,209470000,26.07643,-80.11462,3.6,359.8,4,4.0,3.0
2021-03-05 12:59:55,209470000,26.07758,-80.11457,3.6,3.8,4,4.0,3.0
2021-03-05 13:00:06,209470000,26.07778,-80.11457,3.6,3.9,4,4.0,3.0


In [7]:
df = df.filter(~pl.col('Navigation_sequence').is_in(error_sequence))

In [8]:
# Dropping the sequences which have more than 50 SOG which is abnormal arounud 25 such seq are there comprising 25K datapoints
mask = df.filter(pl.col('SOG') > 50)['Navigation_sequence'].unique()
df = df.filter(~pl.col('Navigation_sequence').is_in(mask))

In [9]:
df

BaseDateTime,MMSI,LAT,LON,SOG,COG,Navigation_sequence,i,j
datetime[μs],i64,f64,f64,f64,f64,i64,f64,f64
2021-02-27 12:52:20,205553000,26.10166,-80.00492,6.5,329.0,2,7.0,15.0
2021-02-27 12:53:29,205553000,26.10342,-80.00615,6.5,328.0,2,7.0,15.0
2021-02-27 12:54:40,205553000,26.10527,-80.00736,6.5,328.1,2,7.0,15.0
2021-02-27 12:55:50,205553000,26.10697,-80.00869,6.2,321.0,2,7.0,14.0
2021-02-27 12:57:00,205553000,26.1085,-80.01017,6.3,317.0,2,8.0,14.0
2021-02-27 12:58:20,205553000,26.11015,-80.01196,6.2,315.0,2,8.0,14.0
2021-02-27 12:59:29,205553000,26.11153,-80.01354,6.2,314.0,2,8.0,14.0
2021-02-27 13:00:10,205553000,26.11231,-80.01447,6.2,312.0,2,8.0,14.0
2021-02-27 13:01:19,205553000,26.11367,-80.01608,6.2,315.0,2,8.0,14.0


In [33]:
df_pandas = df.to_pandas()

In [34]:
df_pandas.shape

(309414, 9)

In [35]:
df_pandas.nunique()

BaseDateTime           306347
MMSI                      357
LAT                      9954
LON                     13120
SOG                       433
COG                      3601
Navigation_sequence      1846
i                          12
j                          16
dtype: int64

In [36]:
df_pandas.dtypes

BaseDateTime           datetime64[ns]
MMSI                            int64
LAT                           float64
LON                           float64
SOG                           float64
COG                           float64
Navigation_sequence             int64
i                             float64
j                             float64
dtype: object

In [37]:
df_pandas.sort_values(by = ['Navigation_sequence','BaseDateTime'], inplace = True)

In [38]:
# Resample the DataFrame with a frequency of 1 minute and use forward fill to fill missing data
df_pandas = (df_pandas.set_index(['BaseDateTime'])).groupby('Navigation_sequence').resample('T').last()

# Reset the index to make timestamp a regular column again
df_pandas = df_pandas.drop(['Navigation_sequence'], axis = 1).reset_index()
df_pandas.ffill(inplace = True)

In [39]:
df_pandas

,Navigation_sequence,BaseDateTime,MMSI,LAT,LON,SOG,COG,i,j
0,2,2021-02-27 12:52:00,205553000.0,26.10166,-80.00492,6.5,329.0,7.0,15.0
1,2,2021-02-27 12:53:00,205553000.0,26.10342,-80.00615,6.5,328.0,7.0,15.0
2,2,2021-02-27 12:54:00,205553000.0,26.10527,-80.00736,6.5,328.1,7.0,15.0
3,2,2021-02-27 12:55:00,205553000.0,26.10697,-80.00869,6.2,321.0,7.0,14.0
4,2,2021-02-27 12:56:00,205553000.0,26.10697,-80.00869,6.2,321.0,7.0,14.0
...,...,...,...,...,...,...,...,...,...
592916,4039,2021-01-30 20:44:00,985380125.0,26.05998,-80.12995,5.9,262.2,2.0,1.0
592917,4039,2021-01-30 20:45:00,985380125.0,26.05998,-80.12995,5.9,262.2,2.0,1.0
592918,4039,2021-01-30 20:46:00,985380125.0,26.06018,-80.13224,2.9,321.7,2.0,1.0
592919,4039,2021-01-30 20:47:00,985380125.0,26.06018,-80.13224,2.9,321.7,2.0,1.0


## We are loosing too much of data due to our assumption maybe
- So as of now lets just use resampled data from previous analysis to regenerate results with polars